# Example of unconstrained mimimization

## Solving a simple problem

In [3]:
include("..\\src\\Polyopt.jl")


Main.Polyopt

We define two variables $x$ and $z$,

In [4]:
x, z = Polyopt.variables(["x", "z"]);

and the function $f(x,z)=4 x^2 + xz - 4z^2 - \frac{21}{10}x^4 + 4z^4 + \frac{1}{3}x^6$.

In [5]:
f = 4*x^2 + x*z - 4*z^2 - 21//10*x^4 + 4*z^4 + 1//3*x^6;

To minimize $f(x,z)$ we form a moment relaxation of order 3,

In [6]:
prob = Polyopt.momentprob(3, f);

and we solve it,

In [7]:
X, Z, t, y, solsta = Polyopt.solve_mosek(prob);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 28              
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                  

The optimal lower bound is

In [8]:
t

-1.031628453600427

If an optimal polynomial solution is found, then $y$ corresponds to the monomial basis vector

In [9]:
[prob.basis y]

28×2 Array{Main.Polyopt.Poly{Float64},2}:
 1.0      1.0
 z        1.43819e-16
 z^2      0.507879
 z^3      7.3042e-17
 z^4      0.257941
 z^5      3.70969e-17
 z^6      2.25121
 x        -1.81306e-17
 x*z      -0.0640265
 x*z^2    -9.20804e-18
 x*z^3    -0.0325178
 x*z^4    -4.67647e-18
 x*z^5    -0.0150525
 ⋮        
 x^2*z^3  5.89547e-19
 x^2*z^4  0.0038416
 x^3      -1.46339e-19
 x^3*z    -0.0005168
 x^3*z^2  -7.43222e-20
 x^3*z^3  -0.000261141
 x^4      6.51605e-5
 x^4*z    9.37036e-21
 x^4*z^2  3.46802e-5
 x^5      -1.18526e-21
 x^5*z    -4.17956e-6
 x^6      5.59344e-7

Because of multiple global minima of $f(x,z)$ this is not the case.

## Perturbing the problem to find global minimum

 We perturb $f(x,z)$ to find a global minimizer,

In [10]:
fp = f + 1e-4*(x+z)

0.0001*z-4.0*z^2+4.0*z^4+0.0001*x+x*z+4.0*x^2-2.1*x^4+0.3333333333333333*x^6

In [11]:
probp = Polyopt.momentprob(3, fp);

In [12]:
Xp, Zp, tp, yp, solsta = Polyopt.solve_mosek(probp);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 28              
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                  

We get (approximately) the same optimal lower bound

In [13]:
tp

-1.0316907357007523

In [14]:
[probp.basis yp]

28×2 Array{Main.Polyopt.Poly{Float64},2}:
 1.0      1.0
 z        -0.712659
 z^2      0.507887
 z^3      -0.36195
 z^4      0.257949
 z^5      -0.18383
 z^6      1.74484
 x        0.0898296
 x*z      -0.0640184
 x*z^2    0.0456233
 x*z^3    -0.0325141
 x*z^4    0.0231715
 x*z^5    -0.0152742
 ⋮        
 x^2*z^3  -0.00292073
 x^2*z^4  0.00344771
 x^3      0.000724872
 x^3*z    -0.000516593
 x^3*z^2  0.000368153
 x^3*z^3  -0.000261158
 x^4      6.51215e-5
 x^4*z    -4.64051e-5
 x^4*z^2  3.43284e-5
 x^5      5.84931e-6
 x^5*z    -4.17325e-6
 x^6      5.46421e-7

Now we have found a global minimizer, and can we verify optimality by evaluating $f(x,z)$ at that point

In [15]:
xo = [yp[8], yp[2]]

2-element Array{Float64,1}:
  0.08982955117272301
 -0.7126589268498063

In [16]:
Polyopt.evalpoly(f, xo)

-1.0316284528008068

## Plotting the solution

In [42]:
using PyPlot

n = 100
X = linspace(-1.5, 1.5, n)
Z = linspace(-1, 1, n)

Xgrid = repmat(X',n,1)
Zgrid = repmat(Z,1,n)
T = zeros(n,n)

for i in 1:n
    for j in 1:n
        T[i:i,j:j] = Polyopt.evalpoly(f, [X[j];Z[i]])
    end
end

fig = figure(figsize=(15,10))
ax = fig[:gca](projection = "3d") 
ax[:plot_surface](Xgrid, Zgrid, T, rstride=2, edgecolors="k", cstride=2, cmap=ColorMap("gray"), linewidth=0.25)
ax[:view_init](20.0, 55.0)
ax[:contour](Xgrid, Zgrid, T, 15, offset=-2.0, colors="black")
ax[:plot]( [xo[1]], [xo[2]], -2.0, "ro")
xlabel("x") 
ylabel("z")


┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1278
ERROR: LoadError: LoadError: PyCall not properly installed. Please run Pkg.build("PyCall")
Stacktrace:
 [1] error(::String) at .\error.jl:33
 [2] top-level scope at C:\Users\SSRL_X1Yoga\.julia\packages\PyCall\zqDXB\src\startup.jl:44
 [3] include(::Function, ::Module, ::String) at .\Base.jl:380
 [4] include at .\Base.jl:368 [inlined]
 [5] include(::String) at C:\Users\SSRL_X1Yoga\.julia\packages\PyCall\zqDXB\src\PyCall.jl:1
 [6] top-level scope at C:\Users\SSRL_X1Yoga\.julia\packages\PyCall\zqDXB\src\PyCall.jl:34
 [7] include(::Function, ::Module, ::String) at .\Base.jl:380
 [8] include(::Module, ::String) at .\Base.jl:368
 [9] top-level scope at none:2
 [10] eval at .\boot.jl:331 [inlined]
 [11] eval(::Expr) at .\client.jl:467
 [12] top-level scope at .\none:3
in expression starting at C:\Users\SSRL_X1Yoga\.julia\packages\PyCall\zqDXB\src\startup.jl:41
in expression starting at C:\Users\SSRL_X1Y

ErrorException: Failed to precompile PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee] to C:\Users\SSRL_X1Yoga\.julia\compiled\v1.5\PyPlot\oatAj_VkVUF.ji.

## Inspecting problem data and validating solutions

The monomial basis vector defining the problem stored is stored in <code>prob.basis</code>,

In [18]:
prob.basis

28-element Array{Main.Polyopt.Poly{Int64},1}:
 1
 z
 z^2
 z^3
 z^4
 z^5
 z^6
 x
 x*z
 x*z^2
 x*z^3
 x*z^4
 x*z^5
 ⋮
 x^2*z^3
 x^2*z^4
 x^3
 x^3*z
 x^3*z^2
 x^3*z^3
 x^4
 x^4*z
 x^4*z^2
 x^5
 x^5*z
 x^6

The coefficients for $f$ are stored in <code>prob.obj</code>,

In [20]:
using LinearAlgebra
dot(prob.obj, prob.basis)

-4//1*z^2+4//1*z^4+x*z+4//1*x^2-21//10*x^4+1//3*x^6

The symmetric matrices defining the moment matrix are stored in <code>prob.mom[1]</code> column by column,

In [21]:
size(prob.mom[1])

(100, 28)

In [22]:
reshape(prob.mom[1]*prob.basis, 10, 10)

10×10 Array{Main.Polyopt.Poly{Int64},2}:
 1      z        z^2      z^3      …  x*z^2    x^2      x^2*z    x^3
 z      z^2      z^3      z^4         x*z^3    x^2*z    x^2*z^2  x^3*z
 z^2    z^3      z^4      z^5         x*z^4    x^2*z^2  x^2*z^3  x^3*z^2
 z^3    z^4      z^5      z^6         x*z^5    x^2*z^3  x^2*z^4  x^3*z^3
 x      x*z      x*z^2    x*z^3       x^2*z^2  x^3      x^3*z    x^4
 x*z    x*z^2    x*z^3    x*z^4    …  x^2*z^3  x^3*z    x^3*z^2  x^4*z
 x*z^2  x*z^3    x*z^4    x*z^5       x^2*z^4  x^3*z^2  x^3*z^3  x^4*z^2
 x^2    x^2*z    x^2*z^2  x^2*z^3     x^3*z^2  x^4      x^4*z    x^5
 x^2*z  x^2*z^2  x^2*z^3  x^2*z^4     x^3*z^3  x^4*z    x^4*z^2  x^5*z
 x^3    x^3*z    x^3*z^2  x^3*z^3     x^4*z^2  x^5      x^5*z    x^6

and similarly the moment matrix evaluated at <code>yp</code> is

In [23]:
M=reshape(prob.mom[1]*yp, 10, 10)

10×10 Array{Float64,2}:
  1.0          -0.712659      0.507887     …  -0.00575074    0.000724872
 -0.712659      0.507887     -0.36195          0.00409835   -0.000516593
  0.507887     -0.36195       0.257949        -0.00292073    0.000368153
 -0.36195       0.257949     -0.18383          0.00344771   -0.000261158
  0.0898296    -0.0640184     0.0456233       -0.000516593   6.51215e-5
 -0.0640184     0.0456233    -0.0325141    …   0.000368153  -4.64051e-5
  0.0456233    -0.0325141     0.0231715       -0.000261158   3.43284e-5
  0.00806942   -0.00575074    0.00409835      -4.64051e-5    5.84931e-6
 -0.00575074    0.00409835   -0.00292073       3.43284e-5   -4.17325e-6
  0.000724872  -0.000516593   0.000368153     -4.17325e-6    5.46421e-7

In [24]:
eigvals(Symmetric(M,:L))

10-element Array{Float64,1}:
 -1.572970461346444e-10
 -1.3245633424888838e-10
  3.40129763387874e-10
  6.775164202499686e-9
  2.0986177927422714e-8
  1.0095377866225661e-7
  2.983768841811685e-6
  0.0013636478363331184
  1.279301207656423
  2.2457282327103076

The (primal) solution is a semidefinite matrix

In [25]:
Xp[1]

10×10 Array{Float64,2}:
  1.03169      5.0e-5       -2.02972      …   1.61009e-6    4.90678e-6
  5.0e-5       0.0594342     1.66473e-9       6.59838e-5   -0.138037
 -2.02972      1.66473e-9    4.0              6.71587e-8   -2.97174e-5
 -1.64045e-9  -1.64208e-8    7.0699e-16      -3.01777e-7   -3.60485e-8
  5.0e-5       0.472062     -2.3808e-5        0.000556943  -1.09757
  0.0279382    2.38261e-5   -0.000109563  …   2.96292e-5   -2.46812e-5
 -1.80409e-8   0.000109713   3.11638e-8       3.60485e-8   -0.000266334
  0.123363    -0.00022846   -0.227397         2.46812e-5    4.06733e-14
  1.61009e-6   6.59838e-5    6.71587e-8       0.000532669  -1.67153e-14
  4.90678e-6  -0.138037     -2.97174e-5      -1.67153e-14   0.333333

In [26]:
eigvals(Xp[1])

10-element Array{Float64,1}:
 3.8318778335459125e-11
 1.5554630145283757e-10
 3.912576349696899e-7
 8.324109709348872e-5
 0.000530368585633566
 0.011411329871909962
 0.03678937636755653
 0.49903572588666717
 4.134571760097218
 5.0438975853336485

If we store <code>Xp[1]</code> in vectorized form (conforming with <code>prob.mom</code>),

In [27]:
xp = vec(Xp[1])

100-element Array{Float64,1}:
  1.0316907351055058
  4.9999992268596276e-5
 -2.0297170774608477
 -1.6404486229619693e-9
  4.9999986492786656e-5
  0.02793817380745451
 -1.804085792409456e-8
  0.12336326977614287
  1.610087937997323e-6
  4.906779559234484e-6
  4.9999992268596276e-5
  0.05943415729640462
  1.6647259736083284e-9
  ⋮
  0.0005326686013626235
 -1.6715340713750928e-14
  4.906779559234484e-6
 -0.1380367850344982
 -2.9717381027328975e-5
 -3.6048451555222184e-8
 -1.097568493916393
 -2.4681165422627605e-5
 -0.0002663341955465123
  4.0673339826663514e-14
 -1.6715340713750928e-14
  0.333333333474375

then we can verify that 

In [41]:
onevec = zeros(28)
onevec[1] = 1
prob.mom[1]'*xp - (prob.obj - tp*onevec)

28-element SparseArrays.SparseVector{Float64,Int64} with 28 stored entries:
  [1 ]  =  -5.95247e-10
  [2 ]  =  0.0001
  [3 ]  =  2.37471e-9
  [4 ]  =  4.85547e-11
  [5 ]  =  -1.04794e-9
  [6 ]  =  1.41398e-15
  [7 ]  =  2.10275e-10
  [8 ]  =  0.0001
  [9 ]  =  -1.95199e-10
  [10]  =  8.26899e-11
        ⋮
  [18]  =  2.10275e-10
  [19]  =  4.6247e-11
  [20]  =  6.30662e-13
  [21]  =  -9.84954e-11
  [22]  =  -5.65148e-17
  [23]  =  6.28805e-10
  [24]  =  2.58053e-12
  [25]  =  2.1027e-10
  [26]  =  8.13467e-14
  [27]  =  -3.34307e-14
  [28]  =  1.41042e-10